# **CREACION** **DE** **RDF**


In [1]:
!pip install morph-kgc

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.3/53.3 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.2/15.2 MB 86.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.4/216.4 kB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 91.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 138.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 87.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 531.9/531.9 kB 59.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.4/113.4 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.7/526.7 kB 56.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 33.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 5.6 MB/s eta 0:00:00
  Attempting uninstall: duckdb
    Found existing installation: duckdb 0.9.2
    Uninstalling duckdb-0.9

In [2]:
import morph_kgc

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
config1="""
          [/content/drive/MyDrive/hands_on_mapping/parking-paris-limpio.csv]
          mappings: /content/drive/MyDrive/hands_on_mapping/mapeo_parking.rml.ttl
       """
config2="""
          [/content/drive/MyDrive/hands_on_mapping/adresse-paris-updated.csv]
          mappings: /content/drive/MyDrive/hands_on_mapping/mapeo_streets.rml.ttl
       """

In [5]:
g1 = morph_kgc.materialize(config1)
g2 = morph_kgc.materialize(config2)
g = g1 +g2

/usr/local/lib/python3.10/dist-packages/morph_kgc/mapping/mapping_parser.py:390: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  fnml_df = fnml_df.applymap(str)
/usr/local/lib/python3.10/dist-packages/morph_kgc/mapping/mapping_partitioner.py:182: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  f"{self.rml_df['mapping_partition'].value_counts()[0]}.")
/usr/local/lib/python3.10/dist-packages/morph_kgc/materializer.py:36: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data = data.applymap(str)
/usr/local/lib/python3.10/dist-packages/morph_kgc/materializer.py:36: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data = data.applymap(str)
/usr/local/lib/python3.10/dist-packages/morph_kgc/materializer.py

In [6]:
v1 = g1.serialize(destination='/content/drive/MyDrive/hands_on_mapping/parkingsRDF.ttl', format="ttl")
v2 = g2.serialize(destination='/content/drive/MyDrive/hands_on_mapping/streetsRDF.ttl', format="ttl")
v = g.serialize(destination='/content/drive/MyDrive/hands_on_mapping/combinedRDF.ttl', format="ttl")
v

<Graph identifier=Ne2287aeda4a9435eacc74993fc904c0e (<class 'rdflib.graph.Graph'>)>

# **QUERYS**

In [7]:
from rdflib import Graph, Namespace, RDF, RDFS
from rdflib.plugins.sparql import prepareQuery

ns = Namespace("http://paris.linkeddata.es/parkings/resource/ont1#")
ns2 = Namespace('http://paris.linkeddata.es/streets/resource/ont1#')

geo = Namespace("http://www.w3.org/2003/01/geo/wgs84_pos#")



### 1. ¿CUANTOS PARKINGS HAY EN TOTAL?

In [8]:
q1 = prepareQuery('''
    SELECT (COUNT(?parking) AS ?count)
    WHERE {
      ?parking rdf:type ns:Parking .
    }
''',
initNs={"rdf": RDF, "ns": ns})


for s in g.query(q1):
    count = s['count']
    print(count)

126


### 2. ¿QUÉ PARKINGS HAY EN EL DISTRITO 8?

In [ ]:
q2 = prepareQuery('''
    SELECT ?name ?parking WHERE {
      ?parking rdf:type ns:Parking ;
               ns:isinDistrict ?district ;
               ns:hasName ?name .
      ?district ns:idDistrict 8 .
    }
''',
initNs={"rdf": RDF, "ns": ns})

for s in g.query(q2):
    name = s['name']
    parking_uri = s['parking']
    print(f"{name}: {parking_uri}")

CHAMPS ELYSEES: http://paris.linkeddata.es/parkings/resource/parking/75108-P-107
CHAMPS ELYSEES PIERRE CHARRON: http://paris.linkeddata.es/parkings/resource/parking/75108-P-125
CONCORDE: http://paris.linkeddata.es/parkings/resource/parking/75108-P-024
ALMA GEORGE V: http://paris.linkeddata.es/parkings/resource/parking/75108-P-110
ETOILE FRIEDLAND: http://paris.linkeddata.es/parkings/resource/parking/75108-P-109
HAUSSMANN BERRI: http://paris.linkeddata.es/parkings/resource/parking/75108-P-031
MADELEINE TRONCHET: http://paris.linkeddata.es/parkings/resource/parking/75108-P-030
FRANCOIS 1ER: http://paris.linkeddata.es/parkings/resource/parking/75108-P-054
HOCHE: http://paris.linkeddata.es/parkings/resource/parking/75108-P-015
SAINT AUGUSTIN: http://paris.linkeddata.es/parkings/resource/parking/75108-P-014
MALESHERBES ANJOU: http://paris.linkeddata.es/parkings/resource/parking/75108-P-002
ROND POINT (GARAGE): http://paris.linkeddata.es/parkings/resource/parking/75108-P-159
GEORGE V: http:/

### 3. ¿CUÁLES SON LOS 5 DISTRITOS CON MÁS PARKINGS?

In [ ]:
q3 = prepareQuery('''
    SELECT ?id (COUNT(?parking) AS ?count)
    WHERE {
       ?parking rdf:type ns:Parking ;
                ns:isinDistrict ?district .
       ?district ns:idDistrict ?id.
    }
    GROUP BY ?district
    ORDER BY DESC(?count)
    LIMIT 5
''',
initNs={"rdf": RDF, "ns": ns})

for s in g.query(q3):
    district = s['id']
    count = s['count']
    print(f'District {district}:  {count} parkings')



District 17:  14 parkings
District 8:  14 parkings
District 16:  13 parkings
District 12:  12 parkings
District 6:  10 parkings


### 4. DEVOLVER EL NOMBRE DEL PARKING Y EL TELÉFONO CORRESPONDIENTE, DADA UNA 'ServiceUrl'.

In [ ]:
q4 = prepareQuery('''
        SELECT ?c ?name ?phone
        WHERE {
            ?p a ns:Parking.
            ?c ns:ServiceUrl 'https://www.saemes.fr/fr/parking/parking-marcadet'.
            ?p ns:canBeContactedWith ?c.
            ?c ns:hasPhoneNumber ?phone.
            ?p ns:hasName ?name.
         }
  ''',
  initNs = {"ns": ns}
)

for s in g.query(q4):
 print(f'Parking:{s.name}, Phone = {s.phone}')

Parking:MARCADET, Phone = 144829999


### 5. ¿EN QUÉ PARKINGS PUEDEN ENTRAR VEHÍCULOS DE MÁS DE 2M DE ALTURA?

In [ ]:
q5 = prepareQuery('''
        SELECT ?name ?height ?id ?tname
        WHERE {
            ?p a ns:Parking;
              ns:isinDistrict ?d;
              ns:isinThoroughfare ?t.
            ?d ns:idDistrict ?id.
            ?t ns:hasName ?tname.
            ?p ns:hasName ?name.
            ?p ns:hasMaxHeight ?height.
            FILTER (?height >= 200)
         }
  ''',
  initNs = {"ns": ns}
)

for s in g.query(q5):
 print(f'-Parking: {s.name}\nHeight:{s.height}, District {s.id}, Thoroughfare: {s.tname}\n')

-Parking: DAMREMONT
Height:220, District 18, Thoroughfare: rue damremont

-Parking: HECTOR MALOT
Height:200, District 12, Thoroughfare: avenue daumesnil

-Parking: VELOSTATION GARE MONTPARNASSE
Height:230, District 15, Thoroughfare: rue de l arrivee

-Parking: MARCADET
Height:200, District 18, Thoroughfare: rue marcadet

-Parking: REUILLY DIDEROT
Height:350, District 12, Thoroughfare: rue de reuilly

-Parking: PEREIRE
Height:200, District 17, Thoroughfare: boulevard pereire

-Parking: VAN GOGH
Height:260, District 12, Thoroughfare: rue de bercy

-Parking: VELOSTATION VAN GOGH
Height:230, District 12, Thoroughfare: rue van gogh

-Parking: RECOLLETS
Height:200, District 10, Thoroughfare: passage des recollets



### 6. Devuelve los parkings con puntos de recarga eléctrica, junto a su capacidad y sus coordenadas.

In [9]:
q6 = prepareQuery('''
    SELECT ?name ?electricCapacity ?lat ?long WHERE {
       ?parking rdf:type ns:Parking ;
                ns:hasName ?name ;
                ns:hasElectricCapacity ?electricCapacity ;
                ns:hasGeoPoint ?geoPoint .
       ?geoPoint geo:lat ?lat ;
                 geo:long ?long .
       FILTER (?electricCapacity > 0)
    }
''',
initNs={"rdf": RDF, "ns": ns, "geo": geo})


total_parkings = len(g.query(q6))
print(f'TOTAL DE PARKINGS CON electricCapacity: {total_parkings}\n')

for s in g.query(q6):
    print(f"-{s.name}: {s.electricCapacity}\n    Geo: ({s.lat}, {s.long})\n")



TOTAL DE PARKINGS CON electricCapacity: 67

-MADELEINE TRONCHET: 465
    Geo: (232.442.949, 4.887.086.907)

-CITROEN CEVENNES: 6
    Geo: (2.276.337, 4.883.875.291)

-PASSY: 3
    Geo: (22.753.247, 4.885.819.322)

-PORTE DE CHAMPERRET: 6
    Geo: (228.784.794, 4.888.630.953)

-PORTE D ORLEANS: 2
    Geo: (232.584.893, 488.205.466)

-FRANCOIS 1ER: 7
    Geo: (230.564.778, 4.886.736.859)

-PORTE DE SAINT CLOUD: 3
    Geo: (225.373.418, 488.387.931)

-PYRAMIDES: 8
    Geo: (233.318.657, 4.886.512.039)

-TERNES: 3
    Geo: (229.685.646, 4.887.815.862)

-BAC MONTALEMBERT: 3
    Geo: (232.697.691, 4.885.659.157)

-PLACE VICTOR HUGO: 6
    Geo: (228.454.072, 4.886.938.969)

-ODEON: 10
    Geo: (234.112.092, 4.885.096.233)

-HOTEL DE VILLE: 27
    Geo: (2.349.561, 4.885.658.712)

-RIVOLI SEBASTOPOL: 1
    Geo: (234.944.833, 4.885.856.491)

-BOURSE: 3
    Geo: (234.077.428, 4.886.963.785)

-FAUBOURG SAINT ANTOINE: 4
    Geo: (237.556.129, 4.885.084.205)

-HAUSSMANN BERRI: 50
    Geo: (230.862.8

### 7. DADO UN  DISTRITO, DAME EL PRECIO DE SUBS DE MOTO Y NON SUBS DE MOTO, NOMBRE PARKING, CALLE Y RATES

In [ ]:
q7 = prepareQuery('''
        SELECT ?name ?tname ?nsr ?moto_30 ?moto_24 ?annualmoto ?monthlymoto
        WHERE {
            ?p a ns:Parking;
              ns:isinDistrict ?d;
              ns:isinThoroughfare ?t;
              ns:hasName ?name;
              ns:hasSubsRate ?sr;
              ns:hasNonSubsRate ?nsr.
            ?d ns:idDistrict 8.
            ?t ns:hasName ?tname.
            ?nsr ns:has30mnMotoRate ?moto_30;
                 ns:has24hMotoRate ?moto_24.
            FILTER (?moto_30 > '0.0')
            ?sr ns:hasAnnualSubsMotoRate ?annualmoto;
                    ns:hasMonthlySubsMotoRate ?monthlymoto.
         }
  ''',
  initNs = {"ns": ns}
)

for s in g.query(q7):
  print(f'Parking: {s.name}\nThoroughfare: {s.tname}')
  print(f'    NonSubs --->    30m Moto: {s.moto_30}           24h Moto: {s.moto_24}')
  print(f'    Subs    --->    Annual Moto: {s.annualmoto}     Monthly Moto: {s.monthlymoto}\n')

Parking: CHAMPS ELYSEES PIERRE CHARRON
Thoroughfare: rue pierre charron
    NonSubs --->    30m Moto: 0.95           24h Moto: 16.65
    Subs    --->    Annual Moto: 1106.0     Monthly Moto: 106.0

Parking: CONCORDE
Thoroughfare: place de la concorde
    NonSubs --->    30m Moto: 0.9           24h Moto: 15.75
    Subs    --->    Annual Moto: 906.0     Monthly Moto: 87.0

Parking: HAUSSMANN BERRI
Thoroughfare: boulevard haussmann
    NonSubs --->    30m Moto: 0.8           24h Moto: 13.5
    Subs    --->    Annual Moto: 899.5     Monthly Moto: 89.0

Parking: MADELEINE TRONCHET
Thoroughfare: place de la madeleine
    NonSubs --->    30m Moto: 0.8           24h Moto: 14.4
    Subs    --->    Annual Moto: 1020.0     Monthly Moto: 102.0

Parking: FRANCOIS 1ER
Thoroughfare: rue francois 1er
    NonSubs --->    30m Moto: 0.8           24h Moto: 13.5
    Subs    --->    Annual Moto: 1050.0     Monthly Moto: 105.0

Parking: SAINT AUGUSTIN
Thoroughfare: rue de laborde
    NonSubs --->    30m Mot

### 8. HALLAR EL NOMBRE DE TODAS LAS CALLES EN LAS QUE HAYA PARKING CON APARCAMIENTO PARA BICICLETAS

(Tarda un poco en ejecutar)

In [10]:
q8 = prepareQuery('''
    SELECT ?tname
    WHERE {
       ?t a ns2:Thoroughfare;
          ns2:hasName ?tname.
       ?p a ns:Parking;
          ns:isinThoroughfare ?pt;
          ns:hasBclCapacity ?bc.
       ?pt ns:hasName ?ptname.
       FILTER(lcase(?tname) = lcase(?ptname) && ?bc > 0)
       } LIMIT 100
''',
initNs={"ns2": ns2, 'ns':ns})
for s in g.query(q8):
    print(f'- {s.tname}')

- PLACE JOFFRE
- PLACE DE LA MADELEINE
- RUE WATT
- RUE LEBLANC
- RUE DE PASSY
- RUE NUNGESSER ET COLI
- AVENUE DE LA PORTE DE CHAMPERRET
- RUE DE CONSTANTINE
- RUE DES PYRAMIDES
- AVENUE VICTOR HUGO
- AVENUE DAUMESNIL
- BOULEVARD HAUSSMANN
- AVENUE FOCH
- RUE VAN GOGH
- RUE LAGRANGE
- RUE DAUBENTON
- RUE DE LOBAU
- RUE DE CHALON
- SQUARE ALBAN SATRAGNE
- AVENUE DE FRIEDLAND
- RUE ABEL GANCE
- RUE DES BATIGNOLLES
- AVENUE PIERRE DE COUBERTIN
- RUE AMBROISE PARE
- RUE DE RIVOLI
- AVENUE DE LA PORTE DE CLIGNANCOURT
- RUE SOUFFLOT
- BOULEVARD VICTOR
- AVENUE DES CHAMPS ELYSEES
- RUE EMILE DURKHEIM
- RUE LECOURBE
- RUE DE BABYLONE
- AVENUE DU GENERAL SARRAIL
- RUE FREMICOURT
- AVENUE DE VILLIERS
- RUE DE LABORDE
- RUE THOMAS MANN
- BOULEVARD MALESHERBES
- BOULEVARD DE LA BASTILLE
- PLACE DU LOUVRE
- RUE DE REUILLY
- RUE DES PETITS HOTELS
- RUE MAYRAN
- RUE DES HALLES
- AVENUE DU MAINE
- RUE CARDINET
